In [88]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Load Datasets

In [89]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [90]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

X_multi, y_multi = load_data('data/tictac_multi.txt')

## Multilayer Perceptron Classifier

In [91]:
def mlp_clf_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    from sklearn.preprocessing import StandardScaler 
    scaler = StandardScaler()
    scaler.fit(X_train) 
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)
    
    # Define and train model
    model = MLPClassifier(max_iter=300,solver='lbfgs', alpha=1e-5)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx

In [92]:
def print_clf_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of Multilayer Perceptron Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [93]:
val_acc_final, test_acc_final, confusion_mtrx_final = mlp_clf_train(X_final, y_final)
print_clf_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

Performance of Multilayer Perceptron Classification on Final Dataset:
Cross Validation Accuracy =  0.9895762132604238
Test Accuracy =  0.9739583333333334
Confusion Matrix:
[[0.925 0.075]
 [0.    1.   ]]


## Single Dataset

In [94]:
import warnings
warnings.filterwarnings('ignore')
val_acc_single, test_acc_single, confusion_mtrx_single = mlp_clf_train(X_single, y_single)
print_clf_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

Performance of Multilayer Perceptron Classification on Single Dataset:
Cross Validation Accuracy =  0.9110687022900764
Test Accuracy =  0.92372234935164
Confusion Matrix:
[[0.932 0.003 0.012 0.015 0.022 0.006 0.    0.003 0.006]
 [0.    0.917 0.03  0.03  0.006 0.012 0.    0.    0.006]
 [0.011 0.005 0.93  0.011 0.016 0.    0.005 0.005 0.016]
 [0.017 0.009 0.009 0.94  0.009 0.009 0.    0.    0.009]
 [0.04  0.02  0.005 0.    0.926 0.01  0.    0.    0.   ]
 [0.026 0.    0.013 0.026 0.    0.921 0.013 0.    0.   ]
 [0.01  0.04  0.01  0.02  0.    0.    0.919 0.    0.   ]
 [0.02  0.04  0.    0.06  0.02  0.04  0.    0.82  0.   ]
 [0.    0.034 0.    0.011 0.    0.    0.    0.022 0.933]]


# Multilayer Perceptron Regressor

In [95]:
def finetune_reg_parameters(X_train, y_train):
    mlp_regressor = MLPRegressor()
    multioutput_regressor = MultiOutputRegressor(mlp_regressor)

    # Define the parameter grid
    param_grid = {'estimator__alpha': [1e-6],
                  'estimator__hidden_layer_sizes': (100,),
                  'estimator__max_iter': [10000]}


    # RandomizedSearchCV to find the best value of k
    randomized_search = RandomizedSearchCV(multioutput_regressor, param_grid, n_iter=4, cv=10, scoring='neg_mean_squared_error')
    randomized_search.fit(X_train, y_train)

    # Get the best value of k
    best_params = randomized_search.best_params_
    print(best_params)
        
    return best_params

In [96]:
def MLP_reg_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
    
    # from sklearn.preprocessing import StandardScaler 
    # scaler = StandardScaler()
    # scaler.fit(X_train) 
    # X_train = scaler.transform(X_train)  
    # X_test = scaler.transform(X_test)
    
    best_params = finetune_reg_parameters(X_train,y_train)

    remove_str = "estimator__"

    for param in list(best_params.keys()):
        if remove_str in param:
            new_param = param.replace(remove_str, "")
            best_params[new_param] = best_params[param]
            del best_params[param]

    # Define and train model
    base_regressor = MLPRegressor(**best_params)
    model = MultiOutputRegressor(base_regressor)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    # val_rmse = cross_val_score(model, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
    # val_rmse = np.mean(val_rmse * -1)  # Convert to positive

    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="r2")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    test_accuracy = model.score(X_test, y_test)

    # Get RMSE
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    return val_accuracy, test_accuracy, rmse

In [97]:
def print_reg_results(val_rmse, test_accuracy, rmse, dataset_name):
    print(f"Performance of MLP Regression on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_rmse)
    print("Test Accuracy = ", test_accuracy)
    print("RMSE = ", rmse)

## Multi Dataset

In [98]:
val_rmse_multi, test_acc_multi, rmse_multi = MLP_reg_train(X_multi, y_multi)
print_reg_results(val_rmse_multi, test_acc_multi, rmse_multi, "Multi Dataset")